<a href="https://colab.research.google.com/github/janbanot/msc-cs-code/blob/main/sem3/MPDG/MPDG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb

# Ustawienie ziarna losowości dla powtarzalności wyników
RANDOM_STATE = 42

In [ ]:
# Wczytanie Danych
# Bezpośredni link do zbioru danych HCV na UCI Repository
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv"

try:
    df = pd.read_csv(url)
    print("Dane wczytane pomyślnie.")
except Exception as e:
    print(f"Błąd wczytywania danych: {e}")

# Usunięcie pierwszej kolumny 'Unnamed: 0'
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [ ]:
# Opis Danych
print(f"\nLiczba obserwacji: {df.shape[0]}")
print(f"Liczba cech: {df.shape[1]}")
print("\nPierwsze 5 wierszy:")
print(df.head())

print("\nInfo o typach danych i brakach:")
print(df.info())

print("\nRozkład zmiennej celu (Category):")
print(df['Category'].value_counts())

In [ ]:
# Preprocessing Danych
# Zmienna 'Sex' (f, m) -> mapujemy na 0 i 1
df['Sex'] = df['Sex'].map({'m': 0, 'f': 1})

# Zmienna celu 'Category' -> Label Encoding
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])
print(f"Mapowanie klas: {dict(zip(le.classes_, range(len(le.classes_))))}")

# Podział na X (cechy) i y (etykiety)
X = df.drop(columns=['Category'])
y = df['Category']

# Podział na zbiór treningowy i testowy
# Używamy stratify=y, ponieważ klasy mogą być nierówne (dużo dawców krwi, mało chorych)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Obsługa brakujących danych (Missing Values)
imputer = SimpleImputer(strategy='median')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

In [ ]:
# Implementacja Metod Zespołowych
results = {}

# METODA 1: BAGGING (Random Forest)
rf = RandomForestClassifier(random_state=RANDOM_STATE)

# Definicja siatki hiperparametrów
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 15]
}

grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train, y_train)

best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)

print(f"Najlepsze parametry RF: {grid_rf.best_params_}")
print(f"Dokładność (Accuracy) RF: {acc_rf:.4f}")
results['Random Forest'] = acc_rf

In [ ]:
# METODA 2: BOOSTING (XGBoost)
# XGBoost wymaga klas od 0 do N-1, co zapewnia LabelEncoder
xgb_clf = xgb.XGBClassifier(
    random_state=RANDOM_STATE,
    eval_metric='mlogloss',
    use_label_encoder=False
)

# Definicja siatki hiperparametrów
param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

grid_xgb = GridSearchCV(xgb_clf, param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)
grid_xgb.fit(X_train, y_train)

best_xgb = grid_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)
acc_xgb = accuracy_score(y_test, y_pred_xgb)

print(f"Najlepsze parametry XGBoost: {grid_xgb.best_params_}")
print(f"Dokładność (Accuracy) XGBoost: {acc_xgb:.4f}")
results['XGBoost'] = acc_xgb

In [ ]:
# METODA 3: STACKING
# Modele bazowe (Level-0).
# Modele SVM czy KNN wymagają skalowania danych.
# Używamy make_pipeline, aby skalowanie odbywało się wewnątrz Stackingu.

estimators = [
    ('dt', DecisionTreeClassifier(max_depth=5, random_state=RANDOM_STATE)),
    ('svm', make_pipeline(StandardScaler(), SVC(probability=False, random_state=RANDOM_STATE))),
    ('knn', make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5)))
]

# Meta-model (Level-1): Regresja Logistyczna
clf_stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5
)

clf_stacking.fit(X_train, y_train)
y_pred_stack = clf_stacking.predict(X_test)
acc_stack = accuracy_score(y_test, y_pred_stack)

print(f"Dokładność (Accuracy) Stacking: {acc_stack:.4f}")
results['Stacking'] = acc_stack

In [ ]:
# Porównanie i Wnioski
results_df = pd.DataFrame(list(results.items()), columns=['Model', 'Accuracy'])
results_df = results_df.sort_values(by='Accuracy', ascending=False)

print(results_df)

# Wizualizacja wyników
plt.figure(figsize=(8, 5))
sns.barplot(x='Model', y='Accuracy', data=results_df, palette='viridis')
plt.title('Porównanie Dokładności Modeli')
plt.ylim(0.8, 1.0) # Skalujemy oś Y, żeby zobaczyć różnice, bo wyniki mogą być wysokie
plt.ylabel('Accuracy')
plt.show()

# Wyświetlenie szczegółowego raportu dla najlepszego modelu
best_model_name = results_df.iloc[0]['Model']
print(f"\nSzczegółowy raport klasyfikacji dla najlepszego modelu ({best_model_name}):")

if best_model_name == 'Random Forest':
    print(classification_report(y_test, y_pred_rf))
elif best_model_name == 'XGBoost':
    print(classification_report(y_test, y_pred_xgb))
else:
    print(classification_report(y_test, y_pred_stack))